In [1]:
import param as pm
import numpy as np
import pandas as pd
import panel as pn
import string
import random
pn.extension('tabulator')
pd.set_option('display.max_columns', 500)

In [2]:
class TECQFSME(pm.Parameterized):
    boost_factor = pm.Number(1, bounds=(0,4), step=0.1, doc="Multiplicative factor to apply to boosting coefficient.")
    dataset  = pm.DataFrame(columns={'amountUSD', 'projectId', 'voter'}, precedence=-1, doc="Dataset of donations. Must contain amountUSD, projectId, and voter columns.")
    matching_pool = pm.Integer(25_000, bounds=(0, 100_000), step=5_000, doc="Matching pool amount.")
    total_donations = pm.Number(0, constant=True, doc="Summation of amountUSD from donations dataset.")
    total_funding_pool = pm.Number(0, constant=True, doc="Summation of matching_pool and total_donations.")
    allocations  = pm.DataFrame(precedence=-1, doc="Percentages allocation table.")
    results  = pm.DataFrame(precedence=-1, doc="Matched and unmatched funding amounts. Allocation percentages times funding amounts.")
    
    def __init__(self, **params):
        super().__init__(**params)
        self.dataset = self.qf(self.dataset)
        self.update()
        
    @staticmethod
    def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
        """
        Takes a specefied column as the donations column. Applies the QF algorithm to produce a new specefied column and intermediate calculation columns.
        """
        df = df.copy(deep=True)
        df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
        df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
        df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
        df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
        df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
        df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
        return df
    
    @staticmethod
    def signal_boost_v1(df, boost_factor, boost_column='amountUSD', new_column_name='amount_boosted'):
        """
        Given a dataset and a specefied column, applies the flag boost algorithm.
        Requires that the dataset contain 'tec_token_flag' and 'tea_flag'.
        """
        df['coefficient'] = 1 + boost_factor * (df['tec_tokens_flag'].astype(int) | df['tea_flag'].astype(int))
        df[new_column_name] = df[boost_column] * df['coefficient']
        return df

    
    @pm.depends('boost_factor', 'matching_pool', watch=True)
    def update(self):
        # Update total donations and funding pool
        with pm.edit_constant(self):
            self.total_donations = self.dataset['amountUSD'].sum()
            self.total_funding_pool = self.matching_pool + self.total_donations
        
        with pm.parameterized.batch_call_watchers(self):
            # Generate and apply the signal boosting coefficient
            self.dataset = self.signal_boost_v1(self.dataset, self.boost_factor, boost_column='amountUSD', new_column_name='amount_boosted')

            # Compute the Boosted Allocation
            self.dataset = self.qf(self.dataset, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')

            # Remove the intermediate steps
            # self.dataset = self.dataset[self.dataset.columns[~self.dataset.columns.str.contains('sqrt')]]

            # Create an allocations table that contains allocation percentages grouped and summed by project. 
            allocation_columns = ['projectId'] + list(self.dataset.columns[self.dataset.columns.str.contains('allocation')])
            self.allocations = self.dataset[allocation_columns].groupby('projectId').sum()

            # Generate the unmatched results table by multiplying allocation percentages by total donations
            unmatched_results = self.total_donations * self.allocations
            
            # Generate the matched results table by multiplying allocation percentages by total donations plus matching pool
            matched_results = self.total_funding_pool * self.allocations
            
            # Merge matched and unmatched results
            self.results = unmatched_results.merge(matched_results, left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
            
            # Sort results by funding amount
            self.results = self.results.sort_values('quadratic_amount_allocation_matched', ascending=False)
            
            # Add some stats to results
            self.results.insert(0, 'Donor Count', df.groupby('projectId')['voter'].nunique())
            self.results.insert(1, 'Mean Donation', df.groupby('projectId')['amountUSD'].mean())
            self.results.insert(2, 'Median Donation', df.groupby('projectId')['amountUSD'].agg(pd.Series.median))

            # Save the boosting percentage stat
            self.results['SME Percentage Boost'] = 100 * ((self.results['quadratic_amount_boosted_allocation_matched'] - self.results['quadratic_amount_allocation_matched']) / self.results['quadratic_amount_allocation_matched'])


Exploring QF with Mock Data.

In [17]:
w = 2
df = pd.DataFrame([list(np.ones(w)) + [w], ['a']*2+['b'], ["0x"+"".join(random.choices(string.hexdigits, k=8)) for _ in range(w+1)]]).T
df.columns = ['amountUSD', 'projectId', 'voter']
df['amountUSD'] = df['amountUSD'].astype(float)
df['tec_tokens_flag'] = 0
df['tea_flag'] = 0
df

,amountUSD,projectId,voter,tec_tokens_flag,tea_flag
0,1.0,a,0x803bEC80,0,0
1,1.0,a,0xBCeF4c26,0,0
2,2.0,b,0xFaeE5972,0,0


In [18]:
t = TECQFSME(dataset=df, matching_pool=0, boost_factor=0)

pn.Row(t)

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=4, name='Boost factor')
        [2] IntSlider(end=100000, name='Matching pool', step=5000)
        [3] FloatInput(description='Summation of a..., disabled=True, name='Total donations', value=4.0)
        [4] FloatInput(description='Summation of m..., disabled=True, name='Total funding pool', value=4.0)

In [19]:
t.dataset

,amountUSD,projectId,voter,tec_tokens_flag,tea_flag,amountUSD_allocation,sqrt(amountUSD),sum(sqrt(amountUSD)),sq(sum(sqrt(amountUSD))),quadratic_amount_allocation,quadratic_amount,coefficient,amount_boosted,amount_boosted_allocation,sqrt(amount_boosted),sum(sqrt(amount_boosted)),sq(sum(sqrt(amount_boosted))),quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,1.0,a,0x803bEC80,0,0,0.25,1.000000,2.000000,4.0,0.4,1.6,1,1.0,0.25,1.000000,2.000000,4.0,0.4,1.6
1,1.0,a,0xBCeF4c26,0,0,0.25,1.000000,2.000000,4.0,0.4,1.6,1,1.0,0.25,1.000000,2.000000,4.0,0.4,1.6
2,2.0,b,0xFaeE5972,0,0,0.50,1.414214,1.414214,2.0,0.2,0.8,1,2.0,0.50,1.414214,1.414214,2.0,0.2,0.8


In [20]:
t.results.T

projectId,a,b
Donor Count,2.0,1.0
Mean Donation,1.0,2.0
Median Donation,1.0,2.0
amountUSD_allocation_unmatched,2.0,2.0
quadratic_amount_allocation_unmatched,3.2,0.8
amount_boosted_allocation_unmatched,2.0,2.0
quadratic_amount_boosted_allocation_unmatched,3.2,0.8
amountUSD_allocation_matched,2.0,2.0
quadratic_amount_allocation_matched,3.2,0.8
amount_boosted_allocation_matched,2.0,2.0


* Number of donors
* Describe Donations (Mean, Median, Mode)
* Expand Dataset across tegr1

In [21]:
df = pd.read_csv('output/TEGR1.csv')

tec_qf_sme = TECQFSME(dataset=df.copy(deep=True))

tec_qf_sme.results.T

projectId,0xe6424ab2,0x4cd41869,0xdd9b885d,0x8d6f0c7b,0xa9bdf738,0xe8249a10,0x64a30a4b,0x97589cd1,0x23387567,0xec026845,0xf1f4942d,0x72b0d6a6,0x5351510d,0x10b3f00e,0xc401c980,0xcf3165f4
Donor Count,29.000000,25.000000,14.000000,24.000000,24.000000,20.000000,17.000000,19.000000,10.000000,17.000000,12.000000,10.000000,8.000000,7.000000,7.000000,5.000000
Mean Donation,21.099676,20.367331,134.753852,9.597053,7.692555,12.548762,21.226625,14.448157,91.271693,9.953153,16.772840,6.995419,6.139782,6.210735,5.301241,2.981766
Median Donation,6.005987,9.859106,18.707399,5.873165,4.740046,5.998856,7.445838,4.740046,6.005987,5.688977,5.747475,5.847482,5.847482,5.998856,5.688977,1.843793
amountUSD_allocation_unmatched,611.890594,509.183286,2021.307778,230.329281,192.313869,263.524009,360.852626,274.514986,1003.988620,169.203600,218.046923,69.954191,49.118253,43.475148,37.108687,14.908828
quadratic_amount_allocation_unmatched,1379.771324,1078.514627,1013.750531,505.888334,476.296440,413.356603,305.351505,299.021282,227.002239,192.269814,116.445568,28.904906,14.242025,9.528204,7.871441,1.505835
amount_boosted_allocation_unmatched,730.818873,343.687871,1635.875869,263.823284,213.111580,269.370444,448.174721,285.035299,1302.095947,177.613917,171.373092,85.064031,51.418201,44.207002,34.562636,13.487914
quadratic_amount_boosted_allocation_unmatched,1524.927106,776.229628,887.319529,579.128987,510.399550,467.209868,360.184006,312.663326,288.275857,197.708737,98.018407,34.811307,14.827779,9.540081,7.166120,1.310392
amountUSD_allocation_matched,3132.149048,2606.410294,10346.681735,1179.010830,984.417323,1348.928192,1847.134471,1405.188871,5139.222650,866.120349,1116.139829,358.081912,251.426790,222.540835,189.952156,76.315394
quadratic_amount_allocation_matched,7062.781288,5520.706798,5189.191974,2589.544079,2438.068922,2115.892128,1563.035020,1530.631836,1161.980352,984.191815,596.062233,147.958597,72.902157,48.773026,40.292376,7.708078
amount_boosted_allocation_matched,3740.919796,1759.271425,8373.730686,1350.460124,1090.876764,1378.854956,2294.119307,1459.040306,6665.176130,909.171127,877.225555,435.426245,263.199778,226.287054,176.919416,69.042010
